# Multi-task recommenders


In the [basic retrieval tutorial](https://github.com/tensorflow/recommenders/examples/tfrs_movielens.ipynb) we built a retrieval system using movie watches as positive interaction signals.

In many applications, however, there are multiple rich sources of feedback to draw upon. For example, an e-commerce site may record user visits to product pages (abundant, but relatively low signal), image clicks, adding to cart, and, finally, purchases. It may even record post-purchase signals such as reviews and returns.

Integrating all these different forms of feedback is critical to building systems that users love to use, and that do not optimize for any one metric at the expense of overall performance.

In this tutorial, we are going to build a multi-objective recommender for Movielens, using both implicit (movie watches) and explicit signals (ratings).

## Imports


Let's first get our imports out of the way.


In [0]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [0]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're going to use the Movielens 100K dataset.

In [0]:
ratings = tfds.load('movie_lens/100k-ratings', split="train")
movies = tfds.load('movie_lens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_id": x["movie_id"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: {
    "movie_id": x["movie_id"],
})

And repeat our preparations for building vocabularies and splitting the data into a train and a test set:

In [0]:
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_ids = movies.batch(100_000).map(lambda x: x["movie_id"])
user_ids = ratings.batch(100_000).map(lambda x: x["user_id"])

unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## A multi-task model

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In this tutorial, we will define our models as before, but instead of having  a single task, we will have two tasks: one that predicts ratings, and one that predicts movie watches.

The user and movie models are as before:

In [0]:
embedding_dimension = 32

# We wrap these in functions this time so that we can run multiple independent
# models later.
def user_model():
  user_features = [tf.feature_column.embedding_column(
          tf.feature_column.categorical_column_with_vocabulary_list(
              "user_id", unique_user_ids),
          embedding_dimension)]

  user_id_input = {"user_id": tf.keras.Input(shape=(), dtype=tf.string, name="user_id")}
  user_embedding = tf.keras.layers.DenseFeatures(user_features, name="user_embedding")(user_id_input)
  return tf.keras.Model(user_id_input, user_embedding, name="user_model")


def movie_model():
  movie_features = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_vocabulary_list(
      "movie_id", unique_movie_ids),
    embedding_dimension)]

  movie_id_input = {"movie_id": tf.keras.Input(shape=(), dtype=tf.string)}
  movie_embedding = tf.keras.layers.DenseFeatures(movie_features)(movie_id_input)
  return tf.keras.Model(movie_id_input, movie_embedding, name="movie_model")

However, now we will have two tasks. The first is the rating task:

In [0]:
tfrs.tasks.RankingTask(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

Its goal is to predict the ratings as accurately as possible.

The second is the retrieval task:

In [0]:
tfrs.tasks.RetrievalTask(
  corpus_metrics=tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128)
  )
)

As before, this task's goal is to predict which movies the user will or will not watch.

### Putting it together

We put it all together in a model class.

The new component here is that - since we have two tasks and two losses - we need to decide on how important each loss is. We can do this by giving each of the losses a weight, and treating these weights as hyperparameters. If we assign a large loss weight to the rating task, our model is going to focus on predicting ratings (but still use some information from the retrieval task); if we assign a large loss weight to the retrieval task, it will focus on retrieval instead.

In [0]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    # Embeddings as before.
    self.movie_model: tf.keras.layers.Layer = movie_model()
    self.user_model: tf.keras.layers.Layer = user_model()

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(128, activation="relu"),
      tf.keras.layers.Dense(1)
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.RankingTask(
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.RetrievalTask(
      corpus_metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(self.movie_model)
      )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model({"user_id": features["user_id"]})
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.movie_model(
        {"movie_id": features["movie_id"]})
    
    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1))
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(labels=features["user_rating"],
                                   predictions=rating_predictions)
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [0]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [0]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [0]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 20s 2s/step - mean_absolute_error: 2.4299 - factorized_top_k_1: 0.0190 - factorized_top_k_1/top_1_categorical_accuracy: 2.8750e-04 - factorized_top_k_1/top_5_categorical_accuracy: 0.0026 - factorized_top_k_1/top_10_categorical_accuracy: 0.0053 - factorized_top_k_1/top_50_categorical_accuracy: 0.0288 - factorized_top_k_1/top_100_categorical_accuracy: 0.0581 - loss: 7.2993
Epoch 2/3
10/10 [==============================] - 19s 2s/step - mean_absolute_error: 0.9325 - factorized_top_k_1: 0.0190 - factorized_top_k_1/top_1_categorical_accuracy: 2.7500e-04 - factorized_top_k_1/top_5_categorical_accuracy: 0.0026 - factorized_top_k_1/top_10_categorical_accuracy: 0.0053 - factorized_top_k_1/top_50_categorical_accuracy: 0.0287 - factorized_top_k_1/top_100_categorical_accuracy: 0.0580 - loss: 1.2623
Epoch 3/3
5/5 [==============================] - 3s 591ms/step - mean_absolute_error: 0.9210 - factorized_top_k_1: 0.0183 - factorized_top_k_1/top_1_c

{'factorized_top_k_1': array([0.00015, 0.0032 , 0.00555, 0.02715, 0.0553 ], dtype=float32),
 'factorized_top_k_1/top_100_categorical_accuracy': 0.05530000105500221,
 'factorized_top_k_1/top_10_categorical_accuracy': 0.005549999885261059,
 'factorized_top_k_1/top_1_categorical_accuracy': 0.0001500000071246177,
 'factorized_top_k_1/top_50_categorical_accuracy': 0.027149999514222145,
 'factorized_top_k_1/top_5_categorical_accuracy': 0.0031999999191612005,
 'loss': 1.2032581567764282,
 'mean_absolute_error': 0.9209953546524048}

The model does OK on predicting ratings (with an MSE of around 0.92), but performs poorly at predicting which movies will be watched or not: its accuracy at 100 is almost 4 times worse than a model trained solely to predict watches.

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

In [0]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [0]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 22s 2s/step - mean_absolute_error: 3.5634 - factorized_top_k_2: 0.0226 - factorized_top_k_2/top_1_categorical_accuracy: 1.2500e-04 - factorized_top_k_2/top_5_categorical_accuracy: 0.0022 - factorized_top_k_2/top_10_categorical_accuracy: 0.0055 - factorized_top_k_2/top_50_categorical_accuracy: 0.0345 - factorized_top_k_2/top_100_categorical_accuracy: 0.0709 - loss: 70288.2195
Epoch 2/3
10/10 [==============================] - 19s 2s/step - mean_absolute_error: 3.5662 - factorized_top_k_2: 0.0974 - factorized_top_k_2/top_1_categorical_accuracy: 0.0010 - factorized_top_k_2/top_5_categorical_accuracy: 0.0151 - factorized_top_k_2/top_10_categorical_accuracy: 0.0330 - factorized_top_k_2/top_50_categorical_accuracy: 0.1610 - factorized_top_k_2/top_100_categorical_accuracy: 0.2770 - loss: 67911.8878
Epoch 3/3
5/5 [==============================] - 3s 653ms/step - mean_absolute_error: 3.5758 - factorized_top_k_2: 0.0658 - factorized_top_k_2/top

{'factorized_top_k_2': array([0.001 , 0.0057, 0.0139, 0.1017, 0.2065], dtype=float32),
 'factorized_top_k_2/top_100_categorical_accuracy': 0.20649999380111694,
 'factorized_top_k_2/top_10_categorical_accuracy': 0.013899999670684338,
 'factorized_top_k_2/top_1_categorical_accuracy': 0.0010000000474974513,
 'factorized_top_k_2/top_50_categorical_accuracy': 0.10170000046491623,
 'factorized_top_k_2/top_5_categorical_accuracy': 0.00570000009611249,
 'loss': 28542.19921875,
 'mean_absolute_error': 3.575779676437378}

We get the opposite result: a model that does well on retrieval, but poorly on predicting ratings.

### Equally-weighted model

Let's now train a model that weighs both tasks equally.

In [0]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [0]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 19s 2s/step - mean_absolute_error: 2.2820 - factorized_top_k_3: 0.0268 - factorized_top_k_3/top_1_categorical_accuracy: 1.6250e-04 - factorized_top_k_3/top_5_categorical_accuracy: 0.0030 - factorized_top_k_3/top_10_categorical_accuracy: 0.0069 - factorized_top_k_3/top_50_categorical_accuracy: 0.0417 - factorized_top_k_3/top_100_categorical_accuracy: 0.0821 - loss: 70193.5263
Epoch 2/3
10/10 [==============================] - 19s 2s/step - mean_absolute_error: 0.9410 - factorized_top_k_3: 0.0995 - factorized_top_k_3/top_1_categorical_accuracy: 0.0012 - factorized_top_k_3/top_5_categorical_accuracy: 0.0156 - factorized_top_k_3/top_10_categorical_accuracy: 0.0338 - factorized_top_k_3/top_50_categorical_accuracy: 0.1627 - factorized_top_k_3/top_100_categorical_accuracy: 0.2843 - loss: 67744.0327
Epoch 3/3
5/5 [==============================] - 3s 654ms/step - mean_absolute_error: 0.8598 - factorized_top_k_3: 0.0638 - factorized_top_k_3/top

{'factorized_top_k_3': array([0.00045, 0.00545, 0.0124 , 0.0985 , 0.20235], dtype=float32),
 'factorized_top_k_3/top_100_categorical_accuracy': 0.20235000550746918,
 'factorized_top_k_3/top_10_categorical_accuracy': 0.012400000356137753,
 'factorized_top_k_3/top_1_categorical_accuracy': 0.00044999999227002263,
 'factorized_top_k_3/top_50_categorical_accuracy': 0.09849999845027924,
 'factorized_top_k_3/top_5_categorical_accuracy': 0.005450000055134296,
 'loss': 28551.40234375,
 'mean_absolute_error': 0.8597880601882935}

The result is a model that does better on both tasks. This is the promise of transfer learning, where jointly training a model on a set of related tasks can result in a better model that training a set of separate estimators.

Of course, this is not true everywhere. We can expect better results when tasks are closely related, or when we can trasnfer knowledge from a data-abundant task (such as clicks) to a related data-sparse task (such as purchases).